# Mining Challenge Sample Notebook

This notebook gives a small example of working with our data.

## Imports and Database Connection

In [4]:
import re
from mongoengine import connect,disconnect
from pycoshark.mongomodels import Project, VCSSystem, Commit, FileAction, Hunk, Refactoring, IssueSystem, Issue, IssueComment, MailingList, Message,TravisBuild, PullRequest,PullRequestReview,PullRequestCommit,PullRequestSystem,TravisJob
from pycoshark.utils import create_mongodb_uri_string


# You may have to update this dict to match your DB credentials
'''
credentials = {'db_user': '',
               'db_password': '',
               'db_hostname': 'localhost',
               'db_port': 27017,
               'db_authentication_database': '',
               'db_ssl_enabled': False}

uri = create_mongodb_uri_string(**credentials)

connect('smartshark_sample', host=uri, alias='default')
'''
disconnect()
connect(host="mongodb://127.0.0.1:27017/smartshark_2_2")

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [358]:
countOtherJob

18

In [365]:
countDistinctIssue = 43690
countCommit = 83361
countBuild = 7196
countJob = 400
countAvailableLog = 400
countDistinctBuild = 169
countPassedJob = 341
countFailedJob = 41
countOtherJob = 18

In [374]:
countNewFeature = 0

'''
countDistinctIssue = 0
countCommit = 0
countBuild = 0
countJob = 0
countAvailableLog = 0
countDistinctBuild = 0
countPassedJob = 0
countFailedJob = 0
countOtherJob = 0
'''

file1 = open("H:\Research\RE22\Result\Bug\logs\logFilePassed.txt", "a", encoding="utf-8")
file2 = open("H:\Research\RE22\Result\Bug\logs\logFileFailed.txt", "a", encoding="utf-8")
file3 = open("H:\Research\RE22\Result\Bug\logs\logFileOther.txt", "a", encoding="utf-8")

for issues in Issue.objects():
        
        
    # considering the issues that are new requirement
    #if issues.issue_type == 'New Feature':
    if issues.issue_type == 'Bug':
               
        countNewFeature  += 1
  
        if countNewFeature<=108000:
            continue

        # multiple commits referencing the same issue
        if Commit.objects(linked_issue_ids=issues.id).count() >0:
            countDistinctIssue +=1
            for eachItemInCommit in Commit.objects(linked_issue_ids=issues.id):
                countCommit += 1
                if (TravisBuild.objects (commit_id = eachItemInCommit.id).count()> 0):
             
                 
                    for eachItemInTravisBuild in TravisBuild.objects(commit_id = eachItemInCommit.id):
                        countBuild += 1

                        if (TravisJob.objects (build_id = eachItemInTravisBuild.id).count()> 0):
                            countDistinctBuild += 1
                            for eachItemInTravisJob in TravisJob.objects(build_id = eachItemInTravisBuild.id):
                                countJob += 1
                               
                                if {"job_log": {"$exists": True}}:
                                    
                                    countAvailableLog += 1
                                    if eachItemInTravisJob.state == "passed":
                                        countPassedJob += 1
                                        file1.write("log " + str(countPassedJob)+" ")
                                        file1.write(eachItemInTravisJob.state)
                                        file1.write("\n")
                                        file1.write(str(eachItemInTravisJob.job_log))
                                        file1.write("*****************\n")
                                    elif eachItemInTravisJob.state == "failed":
                                        countFailedJob += 1
                                        file2.write("log " + str(countFailedJob)+" ")
                                        file2.write(eachItemInTravisJob.state)
                                        file2.write("\n")
                                        file2.write(str(eachItemInTravisJob.job_log))
                                        file2.write("*****************\n")
                                    else:
                                        countOtherJob += 1
                                        file3.write("log " + str(countOtherJob)+" ")
                                        file3.write(eachItemInTravisJob.state)
                                        file3.write("\n")
                                        file3.write(str(eachItemInTravisJob.job_log))
                                        file3.write("*****************\n")
        '''                         
        if countNewFeature == 108000:
            break
        '''                          
        if countNewFeature%500 == 0:
            print("issue traversed")
            print(countNewFeature)

if file1:
    file1.close()
if file2:
    file2.close()
if file3:
    file3.close()

print("*************************************")
print("total new feature")
print(countNewFeature)
print("total distinct issue")
print(countDistinctIssue)
print("total commit related to issues")
print(countCommit)
print("total build count")
print(countBuild)
print("total distinct build")
print(countDistinctBuild)
print("total job count")
print(countJob)
print("total job record with log")
print(countAvailableLog)
print("total passed build job")
print(countPassedJob)
print("total failed build job")
print(countFailedJob)
print("total other build job")
print(countOtherJob)

*************************************
total new feature
108137
total distinct issue
49583
total commit related to issues
94324
total build count
7553
total distinct build
169
total job count
400
total job record with log
400
total passed build job
341
total failed build job
41
total other build job
18


In [ ]:
# log processing for New Features

In [169]:
count = 0
countFailed = 0
countErrors = 0
countSkipped = 0
logCount = 0

temp1 = list()

temp2 = list()

with open("logFileFailed.txt","r", encoding="utf-8") as f:
    
    for line in f:
        if '*****************'  in line:

            
            logCount += 1
            print("log " + str(logCount))
            print("Test failed " +str(countFailed) +" Test with error "+str(countErrors))
            
            print(temp2)
            print(temp1)
            print("\n")
            temp1.clear()
            temp2.clear()
            countFailed = 0
            countErrors = 0
            countSkipped = 0
        if "Tests run" in line:
            count += 1
            #print("line")
            #print(line)
            x = line.rpartition(']')[2].rpartition('Failures')[2][1:]
            temp = ""
            for c in x:
                if c == ',':
                    break
                temp += c
            #print("failed")
            #print(int(temp))  
            countFailed += int(temp)
            x = x.rpartition('Errors')[2]
            temp = ""
            x = x[1:]
            for c in x:
                if c == ',':
                    break
                temp += c
            #print("errored")
            #print(int(temp))  
            countErrors += int(temp)
        
        e = ""
        if "ERROR!" in line:
            #print(line)
           
            xx = next(f)
            xx = next(f)
            #print(xx)
            
            for char in xx:
                if char == ":":
                    break
                e += char
            #print(e)
            temp1.append(e)
            
        
        if "<<< FAILURE!" in line:
            if "in org" not in line:
                #print(line)
                xx = next(f)
                xx = next(f)
                for char in xx:
                    if char == ":":
                        break
                    e += char
            
            temp2.append(e)
            


log 1
Test failed 0 Test with error 3
['']
['java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException']


log 2
Test failed 10 Test with error 2
['', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', '', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError']
['java.text.ParseException', 'java.text.ParseException']


log 3
Test failed 10 Test with error 2
['', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', '', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError']
['java.text.ParseException', 'java.text.ParseException']


log 4
Test failed 0 Test with error 0
[]
[]


log 5
Test failed 0 

In [ ]:
# log processing for Bug

In [375]:
count = 0
countFailed = 0
countErrors = 0
countSkipped = 0
logCount = 0

temp1 = list()

temp2 = list()

with open("H:\Research\RE22\Result\Bug\logs\logFileFailed.txt","r", encoding="utf-8") as f:
    
    for line in f:
        if '*****************'  in line:

            
            logCount += 1
            print("log " + str(logCount))
            print("Test failed " +str(countFailed) +" Test with error "+str(countErrors))
            
            print(temp2)
            print(temp1)
            print("\n")
            temp1.clear()
            temp2.clear()
            countFailed = 0
            countErrors = 0
            countSkipped = 0
        if "Tests run" in line:
            count += 1
            #print("line")
            #print(line)
            x = line.rpartition(']')[2].rpartition('Failures')[2][1:]
            temp = ""
            for c in x:
                if c == ',':
                    break
                temp += c
            #print("failed")
            #print(int(temp))  
            countFailed += int(temp)
            x = x.rpartition('Errors')[2]
            temp = ""
            x = x[1:]
            for c in x:
                if c == ',':
                    break
                temp += c
            #print("errored")
            #print(int(temp))  
            countErrors += int(temp)
        
        e = ""
        if "ERROR!" in line:
            #print(line)
           
            xx = next(f)
            xx = next(f)
            #print(xx)
            
            for char in xx:
                if char == ":":
                    break
                e += char
            #print(e)
            temp1.append(e)
            
        
        if "<<< FAILURE!" in line:
            if "in org" not in line:
                #print(line)
                xx = next(f)
                xx = next(f)
                for char in xx:
                    if char == ":":
                        break
                    e += char
            
            temp2.append(e)
            

log 1
Test failed 10 Test with error 2
['', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', '', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError']
['java.text.ParseException', 'java.text.ParseException']


log 2
Test failed 0 Test with error 5
['', '', '']
['java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException', 'java.text.ParseException', 'java.text.ParseException']


log 3
Test failed 0 Test with error 0
[]
[]


log 4
Test failed 0 Test with error 0
[]
[]


log 5
Test failed 0 Test with error 0
[]
[]


log 6
Test failed 0 Test with error 0
[]
[]


log 7
Test failed 0 Test with error 0
[]
[]


log 8
Test failed 0 Test with error 0
[]
[]


log 9
Test failed 0 Test with error 0
[]
[]


log 10
Test failed 0 Test with error 0
[]
[]


log 11
Test failed 0

In [265]:
countNewFeature = 0
countDistinctIssue = 0
countCommit = 0
countBuild = 0
countJob = 0
countAvailableLog = 0
countDistinctBuild = 0
countPassedJob = 0
countFailedJob = 0
countOtherJob = 0
testTot = 0


f = open("correlation.txt", "w", encoding="utf-8")

try:
    for issues in Issue.objects():


        # considering the issues that are new requirement
        #if issues.issue_type == 'New Feature':
        if issues.issue_type == 'Bug':    
            countNewFeature  += 1
            
            
            #if countNewFeature < 49376:
            #if countNewFeature < 54911: 
            #if countNewFeature < 61393: 
            #if countNewFeature < 67693: 
            #if countNewFeature < 73137:
            #if countNewFeature < 78646:
            #if countNewFeature < 84475:
            #if countNewFeature < 87696:
            #if countNewFeature <   93724:
            #if countNewFeature < 99434:
           
            if countNewFeature < 106536:
                continue
           
            
            commitCountPerIssue = 0
            #print("********issue")
            # multiple commits referencing the same issue
            passedBuildCommit = 0
            failedBuildCommit = 0
            erroredBuildCommit = 0
            startedBuildCommit = 0
            createdBuildCommit = 0
            canceledBuildCommit = 0

            flag = 0
            if Commit.objects(linked_issue_ids=issues.id).count() >0:
                countDistinctIssue +=1

               
                passedBuildCommit = 0
                failedBuildCommit = 0
                erroredBuildCommit = 0
                startedBuildCommit = 0
                createdBuildCommit = 0
                canceledBuildCommit = 0
                
                for eachItemInCommit in Commit.objects(linked_issue_ids=issues.id):
                    countCommit += 1
                    commitCountPerIssue += 1


                    if (TravisBuild.objects (commit_id = eachItemInCommit.id).count()> 0):
                        flag = 1
                        anotherTestTot += TravisBuild.objects (commit_id = eachItemInCommit.id).count()
                        for eachItemInTravisBuild in TravisBuild.objects(commit_id = eachItemInCommit.id):
                            countBuild += 1
                           
                            if eachItemInTravisBuild.state == 'passed':
                                passedBuildCommit += 1
                                testTot += 1
                            elif eachItemInTravisBuild.state == 'failed':
                                failedBuildCommit += 1
                                testTot += 1
                            elif eachItemInTravisBuild.state == 'errored':
                                erroredBuildCommit += 1
                                testTot += 1
                            elif eachItemInTravisBuild.state == 'started':
                                startedBuildCommit += 1
                                testTot += 1
                            elif eachItemInTravisBuild.state == 'created':
                                createdBuildCommit += 1
                                testTot += 1
                            elif eachItemInTravisBuild.state == 'canceled':
                                canceledBuildCommit += 1
                                testTot += 1
                           

            if flag == 1: 

                if countBuild != testTot:
                    print("problem")
                f.write(str(issues.id)+"!"+str(commitCountPerIssue)+"!"+str(issues.priority)+"!"+str(issues.status)+"!"+str(issues.resolution) +"!"+str(passedBuildCommit)+"!"+ str(failedBuildCommit)+"!"+str(erroredBuildCommit)+"!"+str(startedBuildCommit)+"!"+str(createdBuildCommit)+"!"+str(canceledBuildCommit)+"!"+str(countBuild)+"\n") 

            if countNewFeature%500 == 0:
                print("**************issue traversed*************")
                print(countNewFeature)
                #break

except Exception as e:
    print(countNewFeature)
    print(e)

        
if f:          
    f.close()
print("total new feature")
print(countNewFeature)
print("total distinct issue")
print(countDistinctIssue)
print("total commit related to issues")
print(countCommit)
print("total build count")
print(countBuild)
print("total distinct build")
print(countDistinctBuild)


**************issue traversed*************
107000
**************issue traversed*************
107500
**************issue traversed*************
108000
total new feature
108137
total distinct issue
353
total commit related to issues
1237
total build count
0
total distinct build
0


In [327]:
# for improvement

4

In [377]:
countNewFeature = 0
countDistinctIssue = 0
countCommit = 0
countBuild = 0
countJob = 0
countAvailableLog = 0
countDistinctBuild = 0
countPassedJob = 0
countFailedJob = 0
countOtherJob = 0


file1 = open("H:\Research\RE22\Result\Improvement\logs\logFilePassed.txt", "a", encoding="utf-8")
file2 = open("H:\Research\RE22\Result\Improvement\logs\logFileFailed.txt", "a", encoding="utf-8")
file3 = open("H:\Research\RE22\Result\Improvement\logs\logFileOther.txt", "a", encoding="utf-8")

for issues in Issue.objects():
        
        
    # considering the issues that are new requirement
    #if issues.issue_type == 'New Feature':
    if issues.issue_type == 'Improvement':
               
        countNewFeature  += 1
        '''
        if countNewFeature<=108000:
            continue
        '''
        # multiple commits referencing the same issue
        if Commit.objects(linked_issue_ids=issues.id).count() >0:
            countDistinctIssue +=1
            for eachItemInCommit in Commit.objects(linked_issue_ids=issues.id):
                countCommit += 1
                if (TravisBuild.objects (commit_id = eachItemInCommit.id).count()> 0):
             
                 
                    for eachItemInTravisBuild in TravisBuild.objects(commit_id = eachItemInCommit.id):
                        countBuild += 1

                        if (TravisJob.objects (build_id = eachItemInTravisBuild.id).count()> 0):
                            countDistinctBuild += 1
                            for eachItemInTravisJob in TravisJob.objects(build_id = eachItemInTravisBuild.id):
                                countJob += 1
                               
                                if {"job_log": {"$exists": True}}:
                                    
                                    countAvailableLog += 1
                                    if eachItemInTravisJob.state == "passed":
                                        countPassedJob += 1
                                        file1.write("log " + str(countPassedJob)+" ")
                                        file1.write(eachItemInTravisJob.state)
                                        file1.write("\n")
                                        file1.write(str(eachItemInTravisJob.job_log))
                                        file1.write("*****************\n")
                                    elif eachItemInTravisJob.state == "failed":
                                        countFailedJob += 1
                                        file2.write("log " + str(countFailedJob)+" ")
                                        file2.write(eachItemInTravisJob.state)
                                        file2.write("\n")
                                        file2.write(str(eachItemInTravisJob.job_log))
                                        file2.write("*****************\n")
                                    else:
                                        countOtherJob += 1
                                        file3.write("log " + str(countOtherJob)+" ")
                                        file3.write(eachItemInTravisJob.state)
                                        file3.write("\n")
                                        file3.write(str(eachItemInTravisJob.job_log))
                                        file3.write("*****************\n")
                        
        if countNewFeature == 300:
            break
                         
        if countNewFeature%500 == 0:
            print("issue traversed")
            print(countNewFeature)

if file1:
    file1.close()
if file2:
    file2.close()
if file3:
    file3.close()

print("*************************************")
print("total new feature")
print(countNewFeature)
print("total distinct issue")
print(countDistinctIssue)
print("total commit related to issues")
print(countCommit)
print("total build count")
print(countBuild)
print("total distinct build")
print(countDistinctBuild)
print("total job count")
print(countJob)
print("total job record with log")
print(countAvailableLog)
print("total passed build job")
print(countPassedJob)
print("total failed build job")
print(countFailedJob)
print("total other build job")
print(countOtherJob)

*************************************
total new feature
300
total distinct issue
165
total commit related to issues
276
total build count
12
total distinct build
0
total job count
0
total job record with log
0
total passed build job
0
total failed build job
0
total other build job
0


In [14]:
countDistinctIssue = 13976
countCommit = 30611
countBuild = 3961
countJob = 450
countAvailableLog = 450
countDistinctBuild = 173
countPassedJob = 372
countFailedJob = 71
countOtherJob = 7

In [21]:
countNewFeature = 0
'''
countDistinctIssue = 0
countCommit = 0
countBuild = 0
countJob = 0
countAvailableLog = 0
countDistinctBuild = 0
countPassedJob = 0
countFailedJob = 0
countOtherJob = 0
testTot = 0
'''

file1 = open("H:\Research\RE22\Result\Improvement\logs\logFilePassed.txt", "a", encoding="utf-8")
file2 = open("H:\Research\RE22\Result\Improvement\logs\logFileFailed.txt", "a", encoding="utf-8")
file3 = open("H:\Research\RE22\Result\Improvement\logs\logFileOther.txt", "a", encoding="utf-8")

f = open("correlation.txt", "a", encoding="utf-8")

try:
    for issues in Issue.objects():


        # considering the issues that are new requirement
        #if issues.issue_type == 'New Feature':
        if issues.issue_type == 'Improvement':    
            countNewFeature  += 1
           
            
            if countNewFeature <= 46000:
                continue

            
            commitCountPerIssue = 0
            #print("********issue")
            # multiple commits referencing the same issue
            passedBuildCommit = 0
            failedBuildCommit = 0
            erroredBuildCommit = 0
            startedBuildCommit = 0
            createdBuildCommit = 0
            canceledBuildCommit = 0

            flag = 0
            if Commit.objects(linked_issue_ids=issues.id).count() >0:
                countDistinctIssue +=1

               
                passedBuildCommit = 0
                failedBuildCommit = 0
                erroredBuildCommit = 0
                startedBuildCommit = 0
                createdBuildCommit = 0
                canceledBuildCommit = 0
                
                for eachItemInCommit in Commit.objects(linked_issue_ids=issues.id):
                    countCommit += 1
                    commitCountPerIssue += 1


                    if (TravisBuild.objects (commit_id = eachItemInCommit.id).count()> 0):
                        flag = 1
                        #anotherTestTot += TravisBuild.objects (commit_id = eachItemInCommit.id).count()
                        for eachItemInTravisBuild in TravisBuild.objects(commit_id = eachItemInCommit.id):
                            countBuild += 1
                           
                            if eachItemInTravisBuild.state == 'passed':
                                passedBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'failed':
                                failedBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'errored':
                                erroredBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'started':
                                startedBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'created':
                                createdBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'canceled':
                                canceledBuildCommit += 1
                                #testTot += 1
                           
                            if (TravisJob.objects (build_id = eachItemInTravisBuild.id).count()> 0):
                                countDistinctBuild += 1
                                for eachItemInTravisJob in TravisJob.objects(build_id = eachItemInTravisBuild.id):
                                    countJob += 1

                                    if {"job_log": {"$exists": True}}:

                                        countAvailableLog += 1
                                        if eachItemInTravisJob.state == "passed":
                                            countPassedJob += 1
                                            file1.write("log " + str(countPassedJob)+" ")
                                            file1.write(eachItemInTravisJob.state)
                                            file1.write("\n")
                                            file1.write(str(eachItemInTravisJob.job_log))
                                            file1.write("*****************\n")
                                        elif eachItemInTravisJob.state == "failed":
                                            countFailedJob += 1
                                            file2.write("log " + str(countFailedJob)+" ")
                                            file2.write(eachItemInTravisJob.state)
                                            file2.write("\n")
                                            file2.write(str(eachItemInTravisJob.job_log))
                                            file2.write("*****************\n")
                                        else:
                                            countOtherJob += 1
                                            file3.write("log " + str(countOtherJob)+" ")
                                            file3.write(eachItemInTravisJob.state)
                                            file3.write("\n")
                                            file3.write(str(eachItemInTravisJob.job_log))
                                            file3.write("*****************\n")
                        

            if flag == 1: 
                '''
                if countBuild != testTot:
                    print("problem")
                '''
                f.write(str(issues.id)+"!"+str(commitCountPerIssue)+"!"+str(issues.priority)+"!"+str(issues.status)+"!"+str(issues.resolution) +"!"+str(passedBuildCommit)+"!"+ str(failedBuildCommit)+"!"+str(erroredBuildCommit)+"!"+str(startedBuildCommit)+"!"+str(createdBuildCommit)+"!"+str(canceledBuildCommit)+"!"+str(countBuild)+"\n") 

            '''
            if countNewFeature == 46000:
                break
            '''
            if countNewFeature%500 == 0:
                print("**************issue traversed*************")
                print(countNewFeature)
                #break

except Exception as e:
    print(countNewFeature)
    print(e)

        
if f:          
    f.close()
    
if file1:
    file1.close()
if file2:
    file2.close()
if file3:
    file3.close()
    
print("*************************************")
print("total new feature")
print(countNewFeature)
print("total distinct issue")
print(countDistinctIssue)
print("total commit related to issues")
print(countCommit)
print("total build count")
print(countBuild)
print("total distinct build")
print(countDistinctBuild)
print("total job count")
print(countJob)
print("total job record with log")
print(countAvailableLog)
print("total passed build job")
print(countPassedJob)
print("total failed build job")
print(countFailedJob)
print("total other build job")
print(countOtherJob)


**************issue traversed*************
46500
**************issue traversed*************
47000
**************issue traversed*************
47500
**************issue traversed*************
48000
**************issue traversed*************
48500
**************issue traversed*************
49000
**************issue traversed*************
49500
*************************************
total new feature
49512
total distinct issue
23758
total commit related to issues
52646
total build count
4731
total distinct build
173
total job count
450
total job record with log
450
total passed build job
372
total failed build job
71
total other build job
7


In [ ]:
# log processing for Improvement

In [26]:
count = 0
countFailed = 0
countErrors = 0
countSkipped = 0
logCount = 0
totalFailCount = 0
totalErrorCount = 0

temp1 = list()

temp2 = list()

with open("H:\Research\RE22\Result\Improvement\logs\logFileFailed.txt","r", encoding="utf-8") as f:
    
    for line in f:
        if '*****************'  in line:

            
            logCount += 1
            print("log " + str(logCount))
            print("Test failed " +str(countFailed) +" Test with error "+str(countErrors))
            
            print(temp2)
            print(temp1)
            print("\n")
            temp1.clear()
            temp2.clear()
            countFailed = 0
            countErrors = 0
            countSkipped = 0
        if "Tests run" in line:
            count += 1
            #print("line")
            #print(line)
            x = line.rpartition(']')[2].rpartition('Failures')[2][1:]
            temp = ""
            for c in x:
                if c == ',':
                    break
                temp += c
            #print("failed")
            #print(int(temp))  
            countFailed += int(temp)
            totalFailCount += int(temp)
            x = x.rpartition('Errors')[2]
            temp = ""
            x = x[1:]
            for c in x:
                if c == ',':
                    break
                temp += c
            #print("errored")
            #print(int(temp))  
            countErrors += int(temp)
            totalErrorCount += int(temp)
        
        e = ""
        if "ERROR!" in line:
            #print(line)
           
            xx = next(f)
            xx = next(f)
            #print(xx)
            
            for char in xx:
                if char == ":":
                    break
                e += char
            #print(e)
            temp1.append(e)
            
        
        if "<<< FAILURE!" in line:
            if "in org" not in line:
                #print(line)
                xx = next(f)
                xx = next(f)
                for char in xx:
                    if char == ":":
                        break
                    e += char
            
            temp2.append(e)
print("total Fail")
print(totalFailCount)
print("total Error")
print(totalErrorCount)

log 1
Test failed 0 Test with error 5
['', '', '']
['java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException', 'java.text.ParseException', 'java.text.ParseException']


log 2
Test failed 0 Test with error 3
['']
['java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException']


log 3
Test failed 0 Test with error 3
['']
['java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException']


log 4
Test failed 0 Test with error 3
['']
['java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException', 'java.lang.IllegalArgumentException']


log 5
Test failed 10 Test with error 2
['', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', '', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionError', 'java.lang.AssertionErro

In [ ]:
# Dependency

In [27]:
countNewFeature = 0
countDistinctIssue = 0
countCommit = 0
countBuild = 0
countJob = 0
countAvailableLog = 0
countDistinctBuild = 0
countPassedJob = 0
countFailedJob = 0
countOtherJob = 0
testTot = 0


file1 = open("H:\Research\RE22\Result\Dependency\logs\logFilePassed.txt", "a", encoding="utf-8")
file2 = open("H:\Research\RE22\Result\Dependency\logs\logFileFailed.txt", "a", encoding="utf-8")
file3 = open("H:\Research\RE22\Result\Dependency\logs\logFileOther.txt", "a", encoding="utf-8")

f = open("correlation.txt", "a", encoding="utf-8")

try:
    for issues in Issue.objects():


        # considering the issues that are new requirement
        #if issues.issue_type == 'New Feature':
        if issues.issue_type == 'Dependency' or issues.issue_type == 'Dependency upgrade' :    
            countNewFeature  += 1
           
            '''
            if countNewFeature <= 46000:
                continue
            '''
            
            commitCountPerIssue = 0
            #print("********issue")
            # multiple commits referencing the same issue
            passedBuildCommit = 0
            failedBuildCommit = 0
            erroredBuildCommit = 0
            startedBuildCommit = 0
            createdBuildCommit = 0
            canceledBuildCommit = 0

            flag = 0
            if Commit.objects(linked_issue_ids=issues.id).count() >0:
                countDistinctIssue +=1

               
                passedBuildCommit = 0
                failedBuildCommit = 0
                erroredBuildCommit = 0
                startedBuildCommit = 0
                createdBuildCommit = 0
                canceledBuildCommit = 0
                
                for eachItemInCommit in Commit.objects(linked_issue_ids=issues.id):
                    countCommit += 1
                    commitCountPerIssue += 1


                    if (TravisBuild.objects (commit_id = eachItemInCommit.id).count()> 0):
                        flag = 1
                        #anotherTestTot += TravisBuild.objects (commit_id = eachItemInCommit.id).count()
                        for eachItemInTravisBuild in TravisBuild.objects(commit_id = eachItemInCommit.id):
                            countBuild += 1
                           
                            if eachItemInTravisBuild.state == 'passed':
                                passedBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'failed':
                                failedBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'errored':
                                erroredBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'started':
                                startedBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'created':
                                createdBuildCommit += 1
                                #testTot += 1
                            elif eachItemInTravisBuild.state == 'canceled':
                                canceledBuildCommit += 1
                                #testTot += 1
                           
                            if (TravisJob.objects (build_id = eachItemInTravisBuild.id).count()> 0):
                                countDistinctBuild += 1
                                for eachItemInTravisJob in TravisJob.objects(build_id = eachItemInTravisBuild.id):
                                    countJob += 1

                                    if {"job_log": {"$exists": True}}:

                                        countAvailableLog += 1
                                        if eachItemInTravisJob.state == "passed":
                                            countPassedJob += 1
                                            file1.write("log " + str(countPassedJob)+" ")
                                            file1.write(eachItemInTravisJob.state)
                                            file1.write("\n")
                                            file1.write(str(eachItemInTravisJob.job_log))
                                            file1.write("*****************\n")
                                        elif eachItemInTravisJob.state == "failed":
                                            countFailedJob += 1
                                            file2.write("log " + str(countFailedJob)+" ")
                                            file2.write(eachItemInTravisJob.state)
                                            file2.write("\n")
                                            file2.write(str(eachItemInTravisJob.job_log))
                                            file2.write("*****************\n")
                                        else:
                                            countOtherJob += 1
                                            file3.write("log " + str(countOtherJob)+" ")
                                            file3.write(eachItemInTravisJob.state)
                                            file3.write("\n")
                                            file3.write(str(eachItemInTravisJob.job_log))
                                            file3.write("*****************\n")
                        

            if flag == 1: 
                '''
                if countBuild != testTot:
                    print("problem")
                '''
                f.write(str(issues.id)+"!"+str(commitCountPerIssue)+"!"+str(issues.priority)+"!"+str(issues.status)+"!"+str(issues.resolution) +"!"+str(passedBuildCommit)+"!"+ str(failedBuildCommit)+"!"+str(erroredBuildCommit)+"!"+str(startedBuildCommit)+"!"+str(createdBuildCommit)+"!"+str(canceledBuildCommit)+"!"+str(countBuild)+"\n") 

            '''
            if countNewFeature == 46000:
                break
            '''
            if countNewFeature%200 == 0:
                print("**************issue traversed*************")
                print(countNewFeature)
                #break

except Exception as e:
    print(countNewFeature)
    print(e)

        
if f:          
    f.close()
    
if file1:
    file1.close()
if file2:
    file2.close()
if file3:
    file3.close()
    
print("*************************************")
print("total new feature")
print(countNewFeature)
print("total distinct issue")
print(countDistinctIssue)
print("total commit related to issues")
print(countCommit)
print("total build count")
print(countBuild)
print("total distinct build")
print(countDistinctBuild)
print("total job count")
print(countJob)
print("total job record with log")
print(countAvailableLog)
print("total passed build job")
print(countPassedJob)
print("total failed build job")
print(countFailedJob)
print("total other build job")
print(countOtherJob)


**************issue traversed*************
200
**************issue traversed*************
400
**************issue traversed*************
600
**************issue traversed*************
800
**************issue traversed*************
1000
**************issue traversed*************
1200
**************issue traversed*************
1400
**************issue traversed*************
1600
*************************************
total new feature
1798
total distinct issue
318
total commit related to issues
584
total build count
37
total distinct build
12
total job count
22
total job record with log
22
total passed build job
16
total failed build job
6
total other build job
0


In [ ]:
# log processing for dependency

In [28]:
count = 0
countFailed = 0
countErrors = 0
countSkipped = 0
logCount = 0
totalFailCount = 0
totalErrorCount = 0

temp1 = list()

temp2 = list()

with open("H:\Research\RE22\Result\Dependency\logs\logFileFailed.txt","r", encoding="utf-8") as f:
    
    for line in f:
        if '*****************'  in line:

            
            logCount += 1
            print("log " + str(logCount))
            print("Test failed " +str(countFailed) +" Test with error "+str(countErrors))
            
            print(temp2)
            print(temp1)
            print("\n")
            temp1.clear()
            temp2.clear()
            countFailed = 0
            countErrors = 0
            countSkipped = 0
        if "Tests run" in line:
            count += 1
            #print("line")
            #print(line)
            x = line.rpartition(']')[2].rpartition('Failures')[2][1:]
            temp = ""
            for c in x:
                if c == ',':
                    break
                temp += c
            #print("failed")
            #print(int(temp))  
            countFailed += int(temp)
            totalFailCount += int(temp)
            x = x.rpartition('Errors')[2]
            temp = ""
            x = x[1:]
            for c in x:
                if c == ',':
                    break
                temp += c
            #print("errored")
            #print(int(temp))  
            countErrors += int(temp)
            totalErrorCount += int(temp)
        
        e = ""
        if "ERROR!" in line:
            #print(line)
           
            xx = next(f)
            xx = next(f)
            #print(xx)
            
            for char in xx:
                if char == ":":
                    break
                e += char
            #print(e)
            temp1.append(e)
            
        
        if "<<< FAILURE!" in line:
            if "in org" not in line:
                #print(line)
                xx = next(f)
                xx = next(f)
                for char in xx:
                    if char == ":":
                        break
                    e += char
            
            temp2.append(e)
print("total Fail")
print(totalFailCount)
print("total Error")
print(totalErrorCount)

log 1
Test failed 0 Test with error 0
[]
[]


log 2
Test failed 0 Test with error 0
[]
[]


log 3
Test failed 0 Test with error 0
[]
[]


log 4
Test failed 0 Test with error 0
[]
[]


log 5
Test failed 0 Test with error 0
[]
[]


log 6
Test failed 0 Test with error 0
[]
[]


total Fail
0
total Error
0
